## CITIBIKE DOCKING STATION SCORING

#### Citibike has a Bike Angel program where they incentivize people to rebalance their bikes versus rebalancing with trucks. Please develop a new methodology for scoring each of the stations. 

#### Currently there is a set of scores for the morning then another for the evening commute - you can use that or make it more dynamic such as hourly and different by day. 

#### Please draft a write-up of your methodology and results for presentation during your in person interview.

##### http://www.slate.com/blogs/moneybox/2017/02/09/new_york_s_citi_bike_pays_riders_to_make_it_run_better.html

##### http://bikeangels.citibikenyc.com/

## 1. Import Packages

In [17]:
import sqlalchemy as sql
import geopandas as gp
from shapely.geometry import Point
engine = sql.create_engine('postgresql:///qc_capstone_2017')
from __future__ import print_function
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as pl
%pylab inline
import matplotlib
matplotlib.style.use('seaborn-colorblind')
%matplotlib inline
from matplotlib.dates import YearLocator, MonthLocator, DateFormatter
import statsmodels.formula.api as smf
import statsmodels.api as api
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score as rs
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
from datetime import datetime as dt
from datetime import timedelta

Populating the interactive namespace from numpy and matplotlib


## 2. Get Total Citi Bike Station Docks Data and Citi Bike Trips Data

In [18]:
stationinfo = pd.read_json("https://feeds.citibikenyc.com/stations/stations.json")
stationinfo.head()
stationinfo = stationinfo['stationBeanList'].apply(pd.Series)
totaldocks = stationinfo[["id","totalDocks"]]
totaldocks.head()

,id,totalDocks
0,72,39
1,79,33
2,82,27
3,83,62
4,116,39


In [19]:
query = 'select * from citibike_trips order by starttime desc limit 500000'
trip = pd.read_sql(query, con = engine)
trip.sort_values(['starttime'], inplace=True)
trip.head()

,tripduration,starttime,stoptime,startstationid,endstationid,bikeid,usertype,birthyear,gender
499999,754,2017-03-08 16:58:07,2017-03-08 17:10:42,3090,3117,26443,Subscriber,1964.0,2
499998,199,2017-03-08 16:58:09,2017-03-08 17:01:29,379,486,17720,Subscriber,1959.0,1
499997,1106,2017-03-08 16:58:09,2017-03-08 17:16:36,398,398,24918,Subscriber,1985.0,2
499996,364,2017-03-08 16:58:10,2017-03-08 17:04:14,434,368,27163,Subscriber,1983.0,1
499995,1386,2017-03-08 16:58:10,2017-03-08 17:21:17,457,3320,18154,Subscriber,1977.0,1


In [20]:
trip = trip[["starttime","stoptime","startstationid","endstationid"]]
trip.starttime = pd.to_datetime(trip.starttime, errors = 'coerce')
trip['start'] = map(lambda x: str(x.year) + str(x.month).zfill(2) + str(x.day).zfill(2) + str(x.hour).zfill(2) ,trip['starttime'])
trip['end'] = map(lambda x: str(x.year) + str(x.month).zfill(2) + str(x.day).zfill(2) + str(x.hour).zfill(2) ,trip['stoptime'])
trip.head()

,starttime,stoptime,startstationid,endstationid,start,end
499999,2017-03-08 16:58:07,2017-03-08 17:10:42,3090,3117,2017030816,2017030817
499998,2017-03-08 16:58:09,2017-03-08 17:01:29,379,486,2017030816,2017030817
499997,2017-03-08 16:58:09,2017-03-08 17:16:36,398,398,2017030816,2017030817
499996,2017-03-08 16:58:10,2017-03-08 17:04:14,434,368,2017030816,2017030817
499995,2017-03-08 16:58:10,2017-03-08 17:21:17,457,3320,2017030816,2017030817


## 3. Scoring

### 3.1 Data Processing

In [22]:
stations = list(set(trip.startstationid).union(set(trip.endstationid)))
zhanS = {}
for sta in stations:
    zhan = {}
    trip2 = trip[((trip.startstationid == sta) | (trip.endstationid == sta))]
    for i in range(len(trip2)):
        row = trip2.iloc[i,:]
        if sta == row.startstationid:
            zhan[row.start] = zhan.get(row.start, 0) + 1
        if sta == row.startstationid:
            zhan[row.end] = zhan.get(row.end, 0) - 1
    zhanS[sta] = zhanS.get(sta, zhan)

In [23]:
df = pd.DataFrame.from_dict(zhanS)

In [33]:
df.head()

,72,79,82,83,116,119,120,127,128,143,...,3443,3445,3447,3449,3450,3452,3453,3454,3455,3456
2017030816,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017030817,2.0,0.0,0.0,-2.0,1.0,0.0,0.0,4.0,1.0,2.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017030818,-3.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,2.0,-2.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017030819,2.0,2.0,0.0,0.0,-5.0,0.0,-1.0,-3.0,-1.0,0.0,...,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2017030820,-2.0,-3.0,0.0,1.0,2.0,0.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0


In [25]:
df_temp = pd.DataFrame.transpose(df)

In [34]:
df_temp.head()

,2017030816,2017030817,2017030818,2017030819,2017030820,2017030821,2017030822,2017030823,2017030900,2017030901,...,2017040114,2017040116,2017040119,2017040120,2017040213,2017040215,2017040218,2017040318,2017040419,2017040613
72,1.0,2.0,-3.0,2.0,-2.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,1.0,0.0,0.0,2.0,-3.0,1.0,1.0,-2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,2.0,-2.0,0.0,0.0,1.0,0.0,0.0,2.0,-3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,0.0,1.0,4.0,-5.0,2.0,0.0,-2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.2 Score Function

In [27]:
swapDict = lambda x: {value: key for key in x for value in x[key]}
def score(dh, df, w):
    df.fillna(0, inplace =True)
    dt_dh = dt.strptime(dh, '%Y%m%d%H')
    dh3 = (dt_dh-timedelta(hours=3)).strftime('%Y%m%d%H')
    dh2 = (dt_dh-timedelta(hours=2)).strftime('%Y%m%d%H')
    dh1 = (dt_dh-timedelta(hours=1)).strftime('%Y%m%d%H')
    sum3 = df[dh3] + df[dh2] + df[dh1]
    sum3 = sum3.sort_values()
    ind = [0] + list(cumsum(map(lambda x: int(x*len(sum3)), w))[:-1])+ [(len(df)+1)]
    rating = {}
    for n, i in enumerate(range(-3,4)):
        rating[str(i)] = rating.get(str(i), list(sum3.index.values)[ind[n]:(ind[n+1])])
    return rating

### 3.3 Demo

In [28]:
w = [0.05, 0.1, 0.1, 0.5, 0.1, 0.1, 0.05]

for i, dh in enumerate(['2017030821', '2017030822', '2017030823', '2017030900',"2017030901","2017030902","2017030903",\
                       "2017030904","2017030905"]):
    res = swapDict(score(dh, df_temp, w))
    if i==0:
        output = pd.DataFrame(res.items(), columns=['StationID', dh])
    else:
        tmp = pd.DataFrame(res.items(), columns=['StationID', dh])
        output = pd.merge(output, tmp, on = 'StationID')

In [35]:
output.head()

,StationID,2017030821,2017030822,2017030823,2017030900,2017030901,2017030902,2017030903,2017030904,2017030905
0,72,-1,0,-2,0,0,3,-1,-2,-3
1,79,0,0,0,0,-1,0,0,0,-3
2,82,1,0,0,0,0,0,0,0,0
3,83,2,2,2,3,-1,-2,-3,0,0
4,116,2,-2,0,-2,-2,3,0,0,-3


In [31]:
output.to_csv("output.csv")